# Práctica II: Uso de modelos Transformers (Fine-tunning LoRA)

## 0. Carga de librerias

In [181]:
import re

import numpy as np
import pandas as pd

import torch
import evaluate

from datasets import Dataset
from huggingface_hub import list_models, model_info
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from peft import LoraConfig, get_peft_model, TaskType

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from steam_reviews import ReviewLoader

# 1. Carga del dataset
Hemos decidido valorar reseñas para hacer análisis de sentimiento, pero será sobre un videojuego el cual es uno que a uno del grupo le gusta mucho: Geometry Dash. Como en hugging face no hay datos de reseñas de Steam (o al menos, no encontramos), vamos a apoyarnos en una herramienta que nos permitirá extraer reseñas de Steam: steam-reviews.

In [182]:
# Librerias extras a instalar
# !pip install steam-reviews
# !pip install sentencepiece protobuf tiktoken

Ahora, esta es la idea: Vamos a descargar muchas reseñas del juego, vamos a darles un valor (para hacerlo simple, haremos un **análisis de sentimiento, donde solo tendremos 2 valores posibles: Positivo o negativo**). Para ello, necesitamos la ID del juego, el cual es '322170'. Con esto, podemos empezar a usar la librería que descargamos:

In [183]:
# Funcion que devuelve un objeto con reseñas del juego
def obtener_tipo_reseñas(id_juego, tipo_reseña, num_reseñas=10000):

    # Instanciamos el objeto que nos descargara las reseñas
    loader = ReviewLoader()

    # Lo configuramos
    loader.set_language('english')  # idioma de las reseñas
    loader.set_review_type(tipo_reseña)  # tipo de reseñas: positivas o negativas
    loader.set_num_per_page(100)  # cuantas reseñas por pagina

    # Realizamos la descarga de las reseñas
    reviews = loader.load_batch_from_api([id_juego], num=num_reseñas)[0].review_dict()

    # Devolvemos las reseñas
    return reviews

In [184]:
# Configuramos los datos para extraer la informacion de las reseñas
id_geometry = 322170

# Obtenemos las reseñas
positive_reviews = obtener_tipo_reseñas(id_geometry, 'positive')
negative_reviews = obtener_tipo_reseñas(id_geometry, 'negative')

  0%|          | 0/10000 [00:00<?, ?it/s]

Start the request. The game has total 300000 reviews, and will only get 10000 reviews.


  0%|          | 0/10000 [00:00<?, ?it/s]

Start the request. The game has total 300000 reviews, and will only get 10000 reviews.


100%|█████████▉| 9997/10000 [00:34<00:00, 300.19it/s]

Requests count is 100, will wait for 57 seconds.


10097it [01:32, 109.39it/s]                          


Veamos como son las reseñas:

In [185]:
# Muestra informacion de una reseña
def mostrar_info_reseña(reseña):

    # Mostramos la reseña
    print(f'Idioma: {reseña["language"]}')
    print(f'¿Positiva?: {reseña["voted_up"]}')
    print(f'Reseña:\n{reseña["review"]}')

In [186]:
# Extraemos una reseña positiva y otra negativa
first_positive_review = positive_reviews[0]
forst_negative_review = negative_reviews[0]

# Mostramos numero de reseñas
print('=' * 15, 'Informacion reseñas', '=' * 15)
print(f'Total de reseñas: {len(positive_reviews) + len(negative_reviews)}')
print(f'Total reseñas positivas: {len(positive_reviews)}')
print(f'Total reseñas negativas: {len(negative_reviews)}')

# Mostramos las infos basicas de las reseñas
print('=' * 15, 'Informacion reseña positiva', '=' * 15)
mostrar_info_reseña(first_positive_review)
print('=' * 15, 'Informacion reseña negativa', '=' * 15)
mostrar_info_reseña(forst_negative_review)

=============== Informacion reseñas ===============
Total de reseñas: 20000
Total reseñas positivas: 10000
Total reseñas negativas: 10000
=============== Informacion reseña positiva ===============
Idioma: english
¿Positiva?: True
Reseña:
I lost all my gameplay after the update was released (
=============== Informacion reseña negativa ===============
Idioma: english
¿Positiva?: False
Reseña:
bad stupiud baD, O;PTION FOR CHECKPOINTS (SPOILER: THERE IS NON) NOT ADVISED FOR TWITCHY FINGERED PLAYERS OR YOU WILL DIE AND DIE AND DIE FOR 6 YEARS STRAIGHT BECAUSE ITS STUPIUD


Con esto, ya podemos construir un dataframe:

In [187]:
# Creamos el diccionario final
reviews = {'reviews': list(), 'label': list()}

# Juntamos ambas reseñas y formamos un dataframe
for positive_review, negative_review in zip(positive_reviews, negative_reviews):

    # Añadimos la review positiva
    reviews['reviews'].append(positive_review['review'])
    reviews['label'].append(1 if positive_review['voted_up'] else 0)

    # Ahora la negativa
    reviews['reviews'].append(negative_review['review'])
    reviews['label'].append(1 if negative_review['voted_up'] else 0)

# Pasamos esto a un dataframe
reviews_dataframe = pd.DataFrame(reviews)
reviews_dataframe

,reviews,label
0,I lost all my gameplay after the update was re...,1
1,"bad stupiud baD, O;PTION FOR CHECKPOINTS (SPOI...",0
2,i came to this game many times and now my keyb...,1
3,shit fuck this game,0
4,BEST GAME,1
...,...,...
19995,still i don't like this shit game. and i want ...,0
19996,fun.,1
19997,I hate this game,0
19998,love it but i hate that i cant play it on my c...,1


Pero para los modelos de hugging face, hay que pasarlos a un tipo en especial: Dataset.

In [188]:
reviews_hugging_face = Dataset.from_pandas(reviews_dataframe)
reviews_hugging_face

Dataset({
    features: ['reviews', 'label'],
    num_rows: 20000
})

## ¿Por qué este dataset?
Es verdad que no está nada relacionado con lo que estudiamos, pero tener en cuenta que este juego es muy reciente, por lo cual **no hay información sobre estas reseñas en hugging_face.** Es verdad que la complejidad de entrenar el modelo no será mucha, ya que Steam solo tiene 2 rangos de clasificación: Positivo y negativo. Pero hay comentarios que son muy extraños e inimaginables, lo cuál puede suponer un reto a la hora de entrenar el modelo. Más justificación en el PDF.

# 2. Selección de modelos
Es importante elegir modelos **de la familia encoder only,** ya que nuestra tarea es **sentiment analyzer.** No buscamos generar nuevas reseñas, ni tampoco traducirlas o resumirlas. Aunque veremos que se puede generar reseñas, no es el foco principal. Nos dedicaremos a **analizar las reseñas del juego** y para ello solo necesitamos un encoder-only, que nos sacará el sentimiento (hay que entrenarlo).

## 2.1. Búsqueda rápida de modelos
Hugging face da la facilidad de buscar modelos con filtros, por lo cual, buscaremos los mejores modelos encoders (no especializados) para esta tarea:

In [189]:
# Buscamos los modelos
modelos_sa = list(
        list_models(
        filter='fill-mask',
        sort='downloads',
        direction=-1,
        limit=25
    )
)

# Obtenemos los nombres para ver sus infos
nombres_modelos_sa = [nombre.id for nombre in modelos_sa]

Estos son los 25 mejores modelos según 'descargas' para análisis de sentimiento en inglés. Vamos a comparar sus Model Cards solo si no están especializados (y si no los hemos visto en clase):

In [190]:
# Modelos que no podemos usar (vistos en clase, sacado por chatgpt)
modelos_vistos_en_clase = {
    # --- Notebook 4.1: Introducción a Transformers ---
    "bert-base-uncased",       # Usado para ejemplos de Fill-Mask y Tokenización
    "gpt2",                    # Usado para Text Generation
    "t5-small",                # Usado para Traducción/Resumen
    "distilbert-base-uncased", # Mencionado y usado en pipelines
    "distilbert-base-uncased-finetuned-sst-2-english", # Usado implícitamente en el pipeline("sentiment-analysis")

    # --- Notebook 4.2a: Fine-Tuning con LoRA ---
    "distilbert-base-uncased", # ¡CRÍTICO! Este es el modelo base del ejercicio. NO LO USES.
    "roberta-base-openai-detector", # A veces aparece en versiones antiguas de este ejercicio, lo añado por seguridad

    # --- Notebook 4.2b: RAG (Retrieval Augmented Generation) ---
    "sentence-transformers/all-MiniLM-L6-v2", # Usado para Embeddings (Vector DB)
    "google/flan-t5-small",    # Usado como el LLM generador
    "facebook/rag-token-nq",   # A veces mencionado en teoría de RAG
}

In [ ]:
# Funcion para obtener la informacion de un modelo con model info
def obtener_informacion_modelo(id_modelo):

    # Obtenemos la informacion del modelo
    info_modelo = model_info(id_modelo)

    # Preparamos algunos datos
    card_data = getattr(info_modelo, 'card_data', 'None')
    licencia = card_data['license'] if card_data is not None else 'None'

    # Organizamos la informacion en un dict
    informacion = {
        'nombre': getattr(info_modelo, 'id', 'None'),
        'autor': getattr(info_modelo, 'author', 'None'),
        'tarea': getattr(info_modelo, 'pipeline_tag', 'None'),
        'fecha-creacion': getattr(info_modelo, 'created_at', 'None'),
        'fecha-ultima-acualizacion': getattr(info_modelo, 'last_modified', 'None'),
        'descargas': getattr(info_modelo, 'downloads', 'None'),
        'likes': getattr(info_modelo, 'likes', 'None'),
        'tags': ', '.join(getattr(info_modelo, 'tags', 'None'),),
        'licencia': licencia
    }

    return informacion 

In [192]:
# Funcion que evalua si un modelo lo podemos usar
def can_use_model(informacion_modelo):

    # Compromabmos si el nombre contiene distilbert (visto en clase)
    nombre_modelo = informacion_modelo['nombre'].lower().split('/')[1]
    if nombre_modelo in modelos_vistos_en_clase:  # Evitamos usar los modelos vistos en clase
        return False
    
    # Comprobamos si los tags no son solo fill-mask
    if informacion_modelo['tarea'] != 'fill-mask':
        return False

    return True

In [193]:
# Funcion que devuelve un dataframe comparativo
def obtener_dataframe_informativo(modelos, funcion_informacion):

    # Lista de dicts de los modelos
    list_info_modelos = list()

    # Creamos un dataframe para ver la informacion basica de los modelos
    for nombre_modelo in modelos:

        # Obtenemos la informacion del modelo
        informacion_modelo = funcion_informacion(nombre_modelo)

        # Vemos si esta pre-entrenado: en caso de estarlo, descartamos
        if not can_use_model(informacion_modelo):
            continue

        # Añadimos a la lista
        list_info_modelos.append(informacion_modelo)

    # Creamos el dataframe
    comparativa_modelos = pd.DataFrame(list_info_modelos)
    return comparativa_modelos

In [194]:
# Creamos el dataframe
comparativa_modelos = obtener_dataframe_informativo(nombres_modelos_sa, obtener_informacion_modelo)
comparativa_modelos

,nombre,autor,tarea,fecha-creacion,fecha-ultima-acualizacion,descargas,likes,tags,licencia
0,FacebookAI/roberta-large,FacebookAI,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 12:47:04+00:00,14572286,256,"transformers, pytorch, tf, jax, onnx, safetens...",mit
1,FacebookAI/roberta-base,FacebookAI,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 12:39:28+00:00,8977612,534,"transformers, pytorch, tf, jax, rust, safetens...",mit
2,FacebookAI/xlm-roberta-base,FacebookAI,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 12:48:21+00:00,7438379,762,"transformers, pytorch, tf, jax, onnx, safetens...",mit
3,google-bert/bert-base-multilingual-cased,google-bert,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 11:05:41+00:00,5788701,555,"transformers, pytorch, tf, jax, safetensors, b...",apache-2.0
4,google-bert/bert-base-multilingual-uncased,google-bert,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 11:06:00+00:00,4954656,142,"transformers, pytorch, tf, jax, safetensors, b...",apache-2.0
5,nlpaueb/legal-bert-base-uncased,nlpaueb,fill-mask,2022-03-02 23:29:05+00:00,2022-04-28 14:42:50+00:00,4264760,288,"transformers, pytorch, tf, jax, bert, pretrain...",cc-by-sa-4.0
6,google-bert/bert-base-cased,google-bert,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 11:02:26+00:00,3522011,336,"transformers, pytorch, tf, jax, safetensors, b...",apache-2.0
7,FacebookAI/xlm-roberta-large,FacebookAI,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 12:48:30+00:00,3256300,475,"transformers, pytorch, tf, jax, onnx, safetens...",mit
8,emilyalsentzer/Bio_ClinicalBERT,emilyalsentzer,fill-mask,2022-03-02 23:29:05+00:00,2024-12-03 20:22:45+00:00,2921734,400,"transformers, pytorch, tf, jax, bert, fill-mas...",mit
9,facebook/esm2_t33_650M_UR50D,facebook,fill-mask,2022-09-27 14:36:16+00:00,2023-03-21 15:05:12+00:00,2056394,59,"transformers, pytorch, tf, safetensors, esm, f...",mit


Viendo la información básica de los modelos, podemos quedarnos con 2: el de 'FacebookAI/roberta-base' y 'microsoft/deberta-v3-base', ya que estos (según ChatGPT) **no están especializados en algún dominio y tampoco en sentiment-analizer.** De paso, será interesante comparar un modelo 'actual' (última actualización de roberta: 2024) y uno 'viejito' (última actualización de deberta: 2022).

In [195]:
# Funcion que obtiene los parametros del modelo
def obtener_parametros_modelo(nombre_modelo):

    # Obtenemos el modelo
    modelo = AutoModelForSequenceClassification.from_pretrained(nombre_modelo)

    # Obtenemos los parametros del modelo
    parametros_modelo = sum(parametro.numel() for parametro in modelo.parameters())
    return parametros_modelo

Vamos ahora a analizarlos más en profundidad:

In [196]:
# Funcion para obtener la informacion de un modelo con model info
def obtener_informacion_avanzada(id_modelo):
    try:
        # IMPORTANTE: files_metadata=True es necesario para saber el peso de los archivos
        info = model_info(id_modelo, files_metadata=True)
    except Exception as ex:
        return f'Error al buscar el modelo: {ex}'

    # Intentamos obtener la licencia de los tags
    licencia = 'Desconocida'
    if info.tags:
        for tag in info.tags:
            if tag.startswith("license:"):
                licencia = tag.split(":")[1]
                break

    # Si no está en tags, miramos en card_data con seguridad
    if licencia == 'Desconocida':
        card_data = getattr(info, 'card_data', None)
        if card_data and hasattr(card_data, 'get'):
             licencia = card_data.get('license', 'Desconocida')

    # Obtenemos la arquitectura, que estara dentro del diccionario 'config'
    config = getattr(info, 'config', {}) or {}
    arquitectura = config.get('architectures', ['Desconocida'])[0]
    tipo_modelo = config.get('model_type', 'Desconocido')
    
    # Obtenemos el peso del modelo
    peso_bytes = sum(f.size for f in info.siblings if f.size is not None)
    peso_mb = f"{peso_bytes / (1024 * 1024):.2f} MB"
    peso_gb = f"{peso_bytes / (1024 * 1024 * 1024):.2f} GB"

    # Obtenemos las tecnicas de pre-entrenamiento
    tecnicas_detectadas = []
    if info.tags:
        posibles_tecnicas = ['masked-lm', 'causal-lm', 'seq2seq', 'translation', 'token-classification']
        for tag in info.tags:
            if tag in posibles_tecnicas:
                tecnicas_detectadas.append(tag)
    
    str_tecnicas = ", ".join(tecnicas_detectadas) if tecnicas_detectadas else "Ver Model Card (Texto)"

    # Devolvemos todo en un diccionario
    informacion = {
        'nombre': info.id,
        'autor': getattr(info, 'author', 'None'),
        'tarea': getattr(info, 'pipeline_tag', 'None'),
        'fecha-creacion': getattr(info, 'created_at', 'None'),
        'fecha-ultima-acualizacion': getattr(info, 'last_modified', 'None'), 
        'arquitectura_base': arquitectura,
        'tipo_modelo': tipo_modelo,
        'parametros': obtener_parametros_modelo(id_modelo),
        'peso_disco': f"{peso_mb} ({peso_gb})",
        'licencia': licencia,
        'tecnicas_preentrenamiento': str_tecnicas,
        'descargas': info.downloads,
        'tarea_principal': info.pipeline_tag,
        'likes': getattr(info, 'likes', 'None'),
        'tags': ', '.join(getattr(info, 'tags', 'None'),),
        'licencia': licencia
    }

    return informacion

In [197]:
# Comparamos solo esos 2 modelos
modelos_comparar = {'FacebookAI/roberta-base', 'microsoft/deberta-v3-base'}
comparativa_modelos_finales = obtener_dataframe_informativo(modelos_comparar, obtener_informacion_avanzada)
comparativa_modelos_finales

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,nombre,autor,tarea,fecha-creacion,fecha-ultima-acualizacion,arquitectura_base,tipo_modelo,parametros,peso_disco,licencia,tecnicas_preentrenamiento,descargas,tarea_principal,likes,tags
0,FacebookAI/roberta-base,FacebookAI,fill-mask,2022-03-02 23:29:04+00:00,2024-02-19 12:39:28+00:00,RobertaForMaskedLM,roberta,124647170,2684.78 MB (2.62 GB),mit,Ver Model Card (Texto),8977612,fill-mask,534,"transformers, pytorch, tf, jax, rust, safetens..."
1,microsoft/deberta-v3-base,microsoft,fill-mask,2022-03-02 23:29:05+00:00,2022-09-22 12:34:19+00:00,Desconocida,deberta-v2,184423682,1765.66 MB (1.72 GB),mit,Ver Model Card (Texto),1992303,fill-mask,376,"transformers, pytorch, tf, rust, deberta-v2, d..."


**Información básica:**
- Nº de parámetros: Una diferencia normalita (~185M de parámetros contra ~125M de parámetros).
- Arquitectura base: El más pesado (deberta) tiene una arquitectura desconocida (no se pudo encontrar), pero según Copilot, está basada en RoBERTa (basada en BERT) y roberta, en RobertaForMaskedLM.
- Licencia: Ambos tienen la licencia mit (software libre y de código abierto).
- Peso del modelo: 1.72 contra 2.62GB. Curioso porque deberta tiene más parámetros que roberta.
- Técnicas de pre-entrenamiento: Ambos se pre-entrenaron con la técnica de Masked Language Modeling (MLM)


**Información extra recomendada:**
- Autor: microsoft y FacebookAI (Dos grandes empresas).
- Fecha de creación: Aunque ambos son del mismo año y mes (2022-03-02), roberta está más actualizado (2024 vs 2022).
- Descargas: Roberta ha sido probado por mucha más gente (~10M descargas vs ~2M descargas de Deberta).
- Ambos no tienen cabezas especializadas (para sentiment-analizer, clasificación, etc).

En el PDF se explica más detalladamente las comparaciones.

# 3. Fine-tunning con lora
Aplicaremos la técnica de lora para especializar estos modelos sin necesidad de entrenarlos desde cero.

## 3.1. FT para Deberta:

### 3.1.1. Tokenización de los datos
Mediante map (más rápido) y DataCollatorWithPadding (para generar tokens dinámicamente):

In [198]:
# Funcion para aplicar tokenizado eficiente
def tokenizar_datos_deberta(datos):

    # Tokenizamos
    return token_deberta(
        datos['reviews'],
        truncation=True,
        max_length=512
    )

In [199]:
# Obtenemos el modelo
nombre_deberta = 'microsoft/deberta-v3-base'
deberta_model = AutoModelForSequenceClassification.from_pretrained(nombre_deberta, num_labels=2)

# De las 40k de reseñas, dividimos en train y test para simplificar
datos_split_deberta = reviews_hugging_face.train_test_split(test_size=0.2, seed=42)
train_deberta = datos_split_deberta['train']
test_deberta = datos_split_deberta['test']

# Obtenemos su tokenizador
token_deberta = AutoTokenizer.from_pretrained(nombre_deberta)

# Tokenizamos los datos
train_deberta_tokenizado = train_deberta.map(tokenizar_datos_deberta, batched=True, remove_columns='reviews')
test_deberta_tokenizado = test_deberta.map(tokenizar_datos_deberta, batched=True, remove_columns='reviews')

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/manu/Desktop/Programacion/Trabajos ciencia datos/MG/.env/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [200]:
# Creamos la instancia de DataCollator
data_collator_deberta = DataCollatorWithPadding(
    tokenizer=token_deberta,
    padding=True,
    max_length=None,
    pad_to_multiple_of=None
)

### 3.1.2. Preparamos el modelo para aplicarle LoRA
Así no entrenamos todos los parámetros del modelo (~500M) y solo un porcentaje menos.

In [201]:
# Informacion básica de roberta
print(f'Cabezas de cada capa: {deberta_model.config.num_attention_heads}')
print(f'Formato de guardado de pesos: {deberta_model.dtype}')

# Vemos la estructura de roberta
print('Estructura de roberta:\n', deberta_model)  # Esto es especial para saber  como se llaman los vectores q, k y v, a quienes se les aplica esto

Cabezas de cada capa: 12
Formato de guardado de pesos: torch.float32
Estructura de roberta:
 DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (

In [202]:
# Configuramos el LoRA
configuracion_lora_deberta = LoraConfig(
    r=4,  # Es recomendable comenzar con 8
    lora_alpha=32,  # Recomendable el doble o cuadruple que r
    target_modules=['query_proj', 'value_proj'],  # Se lo aplicamos a los vectores q y v
    lora_dropout=0.1,  # El dropout de siempre
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Aplicamos LoRA al modelo
deberta_model_lora = get_peft_model(deberta_model, configuracion_lora_deberta)

# Obtenemos cuantos parametros vamos a entrenar
parametros_entrenar_deberta = sum(parametro.numel() for parametro in deberta_model_lora.parameters() if parametro.requires_grad)

# Vemos cuanto vamos a entrenar
pesos_modelo_deberta = comparativa_modelos_finales[comparativa_modelos_finales['nombre'] == nombre_deberta]['parametros'].values
print(f'Parámetros totales: {pesos_modelo_deberta}')
print(f'Parámetros entrenables: {parametros_entrenar_deberta}')
print(f'Porcentaje de entreno: {(parametros_entrenar_deberta / pesos_modelo_deberta) * 100}')

Parámetros totales: [184423682]
Parámetros entrenables: 148994
Porcentaje de entreno: [0.08078897]


### 3.1.3. Entrenamos el modelo
Lo cual, ¡Solo estaríamos entrenando aproximadamente un 0.16% de los pesos del modelo!

In [203]:
# Métrica de evaluación
accuracy_metric = evaluate.load("accuracy")

# Esta funcion es del notebook de fine tuning de lora, como nos sirve, hemos decidido reusarla. No hate plis :)
def compute_metrics(eval_pred):

    # Sacamos predicciones y reales
    predictions, labels = eval_pred

    # Convertir logits a clases predichas (argmax sobre dimensión de clases)
    predictions = np.argmax(predictions, axis=1)
    
    # Calcular accuracy comparando predicciones con labels verdaderos
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [204]:
# Preparamos los hiperparametros del modelo
args_deberta = TrainingArguments(
    num_train_epochs=3,  # Con 3 ya se muestra mucha potencia, con 4 o mas puede llegar a overfitting
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='no',
    logging_steps=50,
    report_to='none',
    fp16=True
)

# Creamos el trainer (como lo que vamos a entrenar)
deberta_train = Trainer(
    model=deberta_model_lora,
    args=args_deberta,
    train_dataset=train_deberta_tokenizado,
    eval_dataset=test_deberta_tokenizado,
    processing_class=token_deberta,
    data_collator=data_collator_deberta,
    compute_metrics=compute_metrics
)

Ahora entrenamos el transformer con LoRA:

In [205]:
# Entrenamos
deberta_train.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.377600,0.451699,0.842500
2,0.315600,0.412557,0.848750
3,0.381900,0.397587,0.849250


TrainOutput(global_step=6000, training_loss=0.38709563604990643, metrics={'train_runtime': 386.1325, 'train_samples_per_second': 124.31, 'train_steps_per_second': 15.539, 'total_flos': 1642412860862400.0, 'train_loss': 0.38709563604990643, 'epoch': 3.0})

### 3.1.4. Evaluación del entrenamiento
Ahora, vamos a ver algunas métricas con classification_report:

In [206]:
# Predecimos las reseñas de test
pred_proba_deberta = deberta_train.predict(test_deberta_tokenizado)  # Ya usamos esto como validacion, solo sacaremos mas metricas

# Lo convertimos a 1 o 0
pred_deberta = np.argmax(pred_proba_deberta.predictions, axis=1)

# Mostramos metricas mas avanzadas
metricas_deberta_eval = classification_report(y_true=test_deberta['label'], y_pred=pred_deberta)
print(metricas_deberta_eval)

              precision    recall  f1-score   support

           0       0.84      0.87      0.85      2028
           1       0.86      0.83      0.84      1972

    accuracy                           0.85      4000
   macro avg       0.85      0.85      0.85      4000
weighted avg       0.85      0.85      0.85      4000



Vemos que nos da muy buen f1-score, y tambien muy buen accuracy, precision (cuantas reseñas positivas las ha predicho como positivas correctamene) y recall (cuantas reseñas positivas ha detectado). Por lo cual, podemos decir que LoRA funcionó muy bien para esta tarea. Haremos una última prueba poniendo reseñas nosotros: 

In [207]:
# Reseñas inventadas
reseñas_evaluar = {
    'reviews': [
        'This game is amazing, althought the first leves are difficult, but then online levels are incredible, specially gaunlets.',
        'This poor shit is stressfull. I can`t beat Stereo Madness because the fking cube jumped when I clicked. Delete this game please.',
        'Robot took 5 years to give us 2.2, but personally, I think the update is good, but for 5 years, is quite a long time.',
        'This game is amazing, but I am noob :( (phobos is hard)'
    ],
    'label': [1, 0, 0, 1]
}

# Pasamos esto a un dataset de hugging face
reseñas_evaluar_hugging = Dataset.from_dict(reseñas_evaluar)

In [208]:
# Tokenizamos las reseñas
reseñas_eval_token_deberta = reseñas_evaluar_hugging.map(tokenizar_datos_deberta, batched=True, remove_columns='reviews')

# Se lo pasamos al modelo
pred_proba_eval_deberta = deberta_train.predict(reseñas_eval_token_deberta)

# Vemos los resultados
pred_eval_deberta = np.argmax(pred_proba_eval_deberta.predictions, axis=1)
metricas_deberta_eval_personalizado = classification_report(y_true=pred_proba_eval_deberta.label_ids, y_pred=pred_eval_deberta)
print(metricas_deberta_eval_personalizado)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



Es verdad que estas reseñas son muy simples, pero aun así ha sido capaz de predecir correctamente. Personalmente le damos un 9/10. Vamos a probar a su competidor roberta:

## 3.2. FT para Roberta:

### 3.2.1. Repetimos los mismos pasos
No vamos a alargarnos, ya que es hacer lo mismo pero con el otro modelo:

In [209]:
# Funcion para aplicar tokenizado eficiente
def tokenizar_datos_roberta(datos):

    # Tokenizamos
    return token_roberta(
        datos['reviews'],
        truncation=True,
        max_length=512
    )

In [210]:
# Transformacion de los datos
# Obtenemos el modelo
nombre_roberta = 'FacebookAI/roberta-base'
roberta_model = AutoModelForSequenceClassification.from_pretrained(nombre_roberta, num_labels=2)

# De las 40k de reseñas, dividimos en train y test para simplificar
datos_split_roberta = reviews_hugging_face.train_test_split(test_size=0.2, seed=42)
train_roberta = datos_split_roberta['train']
test_roberta = datos_split_roberta['test']

# Obtenemos su tokenizador
token_roberta = AutoTokenizer.from_pretrained(nombre_roberta)

# Tokenizamos los datos
train_roberta_tokenizado = train_roberta.map(tokenizar_datos_roberta, batched=True, remove_columns='reviews')
test_roberta_tokenizado = test_roberta.map(tokenizar_datos_roberta, batched=True, remove_columns='reviews')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [211]:
# Preparando el datacollator
# Creamos la instancia de DataCollator
data_collator_roberta = DataCollatorWithPadding(
    tokenizer=token_roberta,
    padding=True,
    max_length=None,
    pad_to_multiple_of=None
)

In [212]:
# Informacion básica de roberta
print(f'Cabezas de cada capa: {roberta_model.config.num_attention_heads}')
print(f'Formato de guardado de pesos: {roberta_model.dtype}')

# Vemos la estructura de roberta
print('Estructura de roberta:\n', roberta_model)  # Esto es especial para saber  como se llaman los vectores q, k y v, a quienes se les aplica esto

Cabezas de cada capa: 12
Formato de guardado de pesos: torch.float32
Estructura de roberta:
 RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelf

In [213]:
# Informacion básica de roberta
print(f'Cabezas de cada capa: {roberta_model.config.num_attention_heads}')
print(f'Formato de guardado de pesos: {roberta_model.dtype}')

# Vemos la estructura de roberta
print('Estructura de roberta:\n', roberta_model)  # Esto es especial para saber  como se llaman los vectores q, k y v, a quienes se les aplica esto

Cabezas de cada capa: 12
Formato de guardado de pesos: torch.float32
Estructura de roberta:
 RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelf

In [214]:
# Configuramos el LoRA
configuracion_lora_roberta = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=['key', 'query', 'value'],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Aplicamos LoRA a roberta
roberta_model_lora = get_peft_model(roberta_model, configuracion_lora_roberta)

# Obtenemos cuantos parametros vamos a entrenar
parametros_entrenar_roberta = sum(parametro.numel() for parametro in roberta_model_lora.parameters() if parametro.requires_grad)

# Vemos cuanto vamos a entrenar
pesos_modelo_roberta = comparativa_modelos_finales[comparativa_modelos_finales['nombre'] == nombre_roberta]['parametros'].values
print(f'Parámetros totales: {pesos_modelo_roberta}')
print(f'Parámetros entrenables: {parametros_entrenar_roberta}')
print(f'Porcentaje de entreno: {(parametros_entrenar_roberta / pesos_modelo_roberta) * 100}')

Parámetros totales: [124647170]
Parámetros entrenables: 813314
Porcentaje de entreno: [0.65249295]


Resaltar que solo entrenamos aproximadamente un 0.71% de los pesos.

In [215]:
# Preparando el entrenamiento
# Preparamos los hiperparametros del modelo
args_roberta = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='no',
    logging_steps=50,
    report_to='none',
    fp16=True
)

# Creamos el trainer (como lo que vamos a entrenar)
roberta_train = Trainer(
    model=roberta_model_lora,
    args=args_roberta,
    train_dataset=train_roberta_tokenizado,
    eval_dataset=test_roberta_tokenizado,
    processing_class=token_roberta,
    data_collator=data_collator_roberta,
    compute_metrics=compute_metrics
)

In [216]:
# Entrenamiento de roberta
# Entrenamos
roberta_train.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.431600,0.413771,0.835250
2,0.349600,0.398306,0.843250
3,0.386800,0.396490,0.843750


TrainOutput(global_step=6000, training_loss=0.41176177469889325, metrics={'train_runtime': 248.8027, 'train_samples_per_second': 192.924, 'train_steps_per_second': 24.115, 'total_flos': 1774197131178624.0, 'train_loss': 0.41176177469889325, 'epoch': 3.0})

### 3.2.2. Evaluacion del modelo
Vamos a ver si funciona o no, viendo métricas como antes lo hicimos con deberta:

In [217]:
# Predecimos las reseñas de test
pred_proba_roberta = roberta_train.predict(test_roberta_tokenizado)  # Ya usamos esto como validacion, solo sacaremos mas metricas

# Lo convertimos a 1 o 0
pred_roberta = np.argmax(pred_proba_roberta.predictions, axis=1)

# Mostramos metricas mas avanzadas
metricas_roberta_eval = classification_report(y_true=test_roberta['label'], y_pred=pred_roberta)
print(metricas_roberta_eval)

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      2028
           1       0.85      0.83      0.84      1972

    accuracy                           0.84      4000
   macro avg       0.84      0.84      0.84      4000
weighted avg       0.84      0.84      0.84      4000



Ahora con las reseñas inventadas:

In [218]:
# Tokenizamos las reseñas
reseñas_eval_token_roberta = reseñas_evaluar_hugging.map(tokenizar_datos_roberta, batched=True, remove_columns='reviews')

# Se lo pasamos al modelo
pred_proba_eval_roberta = roberta_train.predict(reseñas_eval_token_roberta)

# Vemos los resultados
pred_eval_roberta = np.argmax(pred_proba_eval_roberta.predictions, axis=1)
metricas_roberta_eval_personalizado = classification_report(y_true=pred_proba_eval_roberta.label_ids, y_pred=pred_eval_roberta)
print(metricas_roberta_eval_personalizado)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



Aquí vemos que predice bien las reseñas como deberta. Personalmente le damos un 9/10, porque no hay mucha diferencia en los resultados.

# 4. Comparación de resultados de ambos modelos
Vamos a ver una comparación entre estos 2 modelos, y de paso, explicaremos por qué uno es mejor que otro:

In [219]:
print('=' * 15, 'Metricas para deberta', '=' * 15)
print(metricas_deberta_eval)
print('=' * 15, 'Metricas para roberta', '=' * 15)
print(metricas_roberta_eval)

=============== Metricas para deberta ===============
              precision    recall  f1-score   support

           0       0.84      0.87      0.85      2028
           1       0.86      0.83      0.84      1972

    accuracy                           0.85      4000
   macro avg       0.85      0.85      0.85      4000
weighted avg       0.85      0.85      0.85      4000

=============== Metricas para roberta ===============
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      2028
           1       0.85      0.83      0.84      1972

    accuracy                           0.84      4000
   macro avg       0.84      0.84      0.84      4000
weighted avg       0.84      0.84      0.84      4000



Vemos que empatan en f1 ambos modelos (tanto en f1 macro como weighted, aunque este último no importa mucho al estar **las clases valanceadas**). Pero si somos algo más críticos, vemos que deberta da un poquito mejor f1, aunque insignificante. Lo principal es que, partimos que deberta tiene más parámetros que roberta, y al final, el rendimiento de ambos es por decirlo igual.

In [220]:
# Imprimimos el numero de parametros entrenados por cada uno
print('=' * 15, 'Parametros deberta', '=' * 15)
print(f'Parámetros entrenables: {parametros_entrenar_deberta}')
print(f'Porcentaje de entreno: {(parametros_entrenar_deberta / pesos_modelo_deberta) * 100}')

print('=' * 15, 'Parametros roberta', '=' * 15)
print(f'Parámetros entrenables: {parametros_entrenar_roberta}')
print(f'Porcentaje de entreno: {(parametros_entrenar_roberta / pesos_modelo_roberta) * 100}')

=============== Parametros deberta ===============
Parámetros entrenables: 148994
Porcentaje de entreno: [0.08078897]
=============== Parametros roberta ===============
Parámetros entrenables: 813314
Porcentaje de entreno: [0.65249295]


En general, entrenamos menos parametros en deberta, pero investigando, vimos que es porque en Roberta, AutoModelForSequenceClassification 'elimina' una de sus capas y le pone una nueva, la cual es la que usamos para clasificar. En cambio, con Deberta, aprovecha dicha capa y añade la nueva de clasificación. En este caso, no pasó esto, pero podría pasar (seguramente si ejecutan la celda de nuevo) que roberta tenga peores resultdaos (mínimos) que deberta:

In [221]:
print('=' * 15, 'Metricas para deberta', '=' * 15)
print(metricas_deberta_eval_personalizado)
print('=' * 15, 'Metricas para roberta', '=' * 15)
print(metricas_roberta_eval_personalizado)

=============== Metricas para deberta ===============
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

=============== Metricas para roberta ===============
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



Vamos a explicar a detalle los resultados de cada uno:

- En cuanto a las métricas con el test, Deberta tiene mejores resultados, dando mejor score a la clase 1 (0.85) que la 0 (0.84). Esto se debe principalmente porque Deberta, se basa en la idea de **mejorar BERT, porque está mal entrenado,** introduciendo algo llamado 'atención desempaquetada,' lo cual, a grandes rasgos, trata de separar los contenidos de la posición de las palabras.
- En cuanto a los pesos entrenados, Deberta vuelve a ganar, ya que, al tener más parámetros/pesos/ LoRA hace que, al crear matrices más pequeñas, se utilize un porcentaje, que a lo mejor es similar a Roberta, pero al existir más parámetros/pesos, pues ahí saca la diferencia (0.08% de Deberta vs 0.6% de Roberta, que aún así es poquísimo).
- Y en cuanto a las métricas de las reseñas personalizadas, pasa lo mismo que con las métricas de test: Deberta pretende arreglar BERT (o ROBERTA), y de ahñi es que su 'mejora' de mejores resultados en general.

Esto se explica mejor en la memoria.

# 5. Extra: Comparación VS otras técnicas (como el prompt engineering)
Vamos a ver si vale la pena entrenar estos modelos en frente a otras opciones más asequibles como aplicar técnicas de **prompt engineering**.

In [222]:
# Modelo que vamos a usar
mejores_modelos_decoder = list(
        list_models(
        filter=['text-generation', 'en'],
        sort='downloads',
        direction=-1,
        limit=10
    )
)

# Sacamos informacion importante
info_modelos_genrativos = {
    'nombre': [modelo.id for modelo in mejores_modelos_decoder],
    'autor': [modelo.author for modelo in mejores_modelos_decoder],
    'descargas': [modelo.downloads for modelo in mejores_modelos_decoder],
    'funcion': [modelo.pipeline_tag for modelo in mejores_modelos_decoder],
    'tags': [modelo.tags for modelo in mejores_modelos_decoder]
}

# Mostramos en un dataframe
df_modelos_decoder = pd.DataFrame(info_modelos_genrativos)
df_modelos_decoder

,nombre,autor,descargas,funcion,tags
0,openai-community/gpt2,None,9934564,text-generation,"[transformers, pytorch, tf, jax, tflite, rust,..."
1,Qwen/Qwen2.5-7B-Instruct,None,7812528,text-generation,"[transformers, safetensors, qwen2, text-genera..."
2,Qwen/Qwen2.5-3B-Instruct,None,6100885,text-generation,"[transformers, safetensors, qwen2, text-genera..."
3,meta-llama/Llama-3.1-8B-Instruct,None,5221966,text-generation,"[transformers, safetensors, llama, text-genera..."
4,Qwen/Qwen2.5-1.5B-Instruct,None,5087315,text-generation,"[transformers, safetensors, qwen2, text-genera..."
5,facebook/opt-125m,None,4492941,text-generation,"[transformers, pytorch, tf, jax, opt, text-gen..."
6,meta-llama/Llama-3.2-1B-Instruct,None,3686452,text-generation,"[transformers, safetensors, llama, text-genera..."
7,Gensyn/Qwen2.5-0.5B-Instruct,None,3572773,text-generation,"[transformers, safetensors, qwen2, text-genera..."
8,meta-llama/Llama-3.2-1B,None,3275581,text-generation,"[transformers, safetensors, llama, text-genera..."
9,Qwen/Qwen2.5-Coder-0.5B-Instruct,None,3142890,text-generation,"[transformers, safetensors, qwen2, text-genera..."


Vamos a quedarnos con Qwen2.5-7B-Instruct, según chatgpt es el mejor candidato para prompt engineering:

**Mejor equilibrio (para reseñas coloquiales con prompt engineering):**

- Qwen2.5-7B-Instruct → potencia y buena comprensión de inglés coloquial.

In [223]:
# Nos vamos a quedar co
modelo_usar = 'Qwen/Qwen2.5-7B-Instruct'
token_decoder = AutoTokenizer.from_pretrained(modelo_usar)
token_decoder.pad_token = token_decoder.eos_token

## 5.1. Zero-shot
Vamos a coger un modelo generativo tipo decoder, y vamos a aplicarle la técnica de prompt engineering (sin poner ejemplos), y veremos que tal le va contra el dataset y las reseñas inventadas por nosotros:

In [224]:
# Vamos a obtener las reseñas
reseñas_usar = reviews_dataframe['reviews'].iloc[:60]  # Solo 60 porque sino se tarda mucho
label = reviews_dataframe['label'].iloc[:60]

Vamos a configurarlo para poder usarlo.

In [225]:
# Configuramos el modelo
zero_shot = pipeline(
    model=modelo_usar,
    tokenizer=token_decoder,
    task='text-generation',
    device=-1
)

# Configuramos el modelo
configuracion_zero_shot = {
    'max_new_tokens': 45,
    'temperature': 0.05,
    'top_k': 10,
    'top_p': 0.8
}

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


Vamos a ver que resultados nos genera:

In [226]:
# Funcion para obtener las respuestas
def obtener_predicciones_chunk(chunk_reseñas, samples={}):

    # Las pasamos a string
    reseñas_chunk_string = '\n'.join(['\t- ' + reseña.replace('\n', '') for reseña in chunk_reseñas])

    # Si queremos ejemplos, los configuramos para metelo al prompt
    if len(samples) != 0:
        samples = '' + '\n'.join([f'- Review: {fila["reviews"]}, Prediction: {fila["label"]}' for fila in samples])
        samples = f'''

    ### SOME EXAMPLES
    {samples}

    ---
    '''
    
    else:
        samples = ''

    # Prompt a usar
    prompt_usar = f"""\n
    =========PROMPT============
    You are a strict binary classification system. Follow the rules EXACTLY.

    ### TASK
    Classify the following {len(chunk_reseñas)} reviews into:
    - `0` = negative
    - `1` = positive

    ### INPUT (exactly {len(chunk_reseñas)} reviews):
    {reseñas_chunk_string}

    ---
    {samples}
    ### RULES (MUST FOLLOW)
    - Output MUST contain **exactly {len(chunk_reseñas)} values**.
    - Each value MUST be only `0` or `1`.
    - Values MUST be separated by commas with NO spaces.
    - NO text before.
    - NO text after.
    - NO explanation.
    - NO formatting.
    - NO markdown.
    - NO additional symbols.
    - NO quotes.

    ---

    ### FINAL INSTRUCTION
    Respond ONLY with the {len(chunk_reseñas)} comma-separated digits. NOTHING ELSE.\n
    =========END of PROMPT============
    """

    # Pasamos esto al modelo
    resultado_zero_shot = zero_shot (
        prompt_usar,
        num_return_sequences=1,
        **configuracion_zero_shot
    )

    # Obtenemos la respuesta
    resultado_generado_zero_shot = resultado_zero_shot[0]['generated_text'].strip()

    # Extraemos la lista de predicciones
    match = re.search(r'[01](?:,[01])+', resultado_generado_zero_shot)

    # Vemos que nos la haya devuelto bien
    if match:
        pred_list = list(map(int, match.group().split(',')))
        return pred_list

    else:
        print(f'Fallo: {resultado_generado_zero_shot}')
        return None

In [227]:
# Funcion que se encarga que se devuelva si o si un vector de predicciones
def obtener_predicciones_chunk_correctas(chunk_reseñas, samples={}):

    # Obtenemos los resultados
    resultados_chunk = obtener_predicciones_chunk(chunk_reseñas, samples)

    # Nos aseguramos que no sea None
    correcto = resultados_chunk is not None
    while not correcto:
        resultados_chunk = obtener_predicciones_chunk(chunk_reseñas, samples)
        correcto = resultados_chunk is not None

    # Devolvemos el vector
    return resultados_chunk

In [ ]:
# Obtenemos los resultados
resultados = list()
chunk = 15

# Pasamos las reseñas en chunks
for i in range(0, reseñas_usar.shape[0], chunk):

    # Sacamos el chunk
    reseñas_chunk = reseñas_usar.iloc[i:i+chunk]

    # Pedimos respuesta al modelo
    prediccion_completa = False
    lista_prediccion_zero_shot = obtener_predicciones_chunk_correctas(reseñas_chunk)
    print(f'Nuevas predicciones del chunk {i}: {lista_prediccion_zero_shot}')

    # Actualizamos lista
    resultados += lista_prediccion_zero_shot

# Mostramos el resultado final
print(resultados)

Nuevas predicciones del chunk 0: [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0]
Nuevas predicciones del chunk 15: [0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Nuevas predicciones del chunk 30: [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
Nuevas predicciones del chunk 45: [1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]


Vamos a ver como le fue:

In [229]:
print(classification_report(y_true=label, y_pred=resultados))

              precision    recall  f1-score   support

           0       0.64      0.83      0.72        30
           1       0.76      0.53      0.63        30

    accuracy                           0.68        60
   macro avg       0.70      0.68      0.68        60
weighted avg       0.70      0.68      0.68        60



Y con nuestro dataset inventado:

In [230]:
# Pedimos respuesta al modelo
lista_prediccion_zero_shot = obtener_predicciones_chunk_correctas(reseñas_evaluar['reviews'])

# Mostramos el resultado final
print(lista_prediccion_zero_shot)

# Mostramos las metricas
print(classification_report(y_true=reseñas_evaluar['label'], y_pred=lista_prediccion_zero_shot))

[1, 0, 1, 1]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.67      1.00      0.80         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



Pues vemos que en verdad, le fue algo mal (o no servimos para prompt engineering). En conclusión, con Zero-shot, vale la pena entrenar el modelo. Vamos a ver ahora con few shot:

## 5.2. Few-shot
Vamos a aplicar la técnica de prompt engineering (poner algunos ejemplos), y veremos que tal le va contra el dataset y las reseñas inventadas por nosotros:

In [235]:
# Sacamos algunos ejemplos
n_samples = 5  # Esto se puede cambiar
examples = reviews_dataframe.iloc[:n_samples, :]

# Lo pasamos a dict
examples_few_shot = examples.to_dict(orient="records")
examples_few_shot

[{'reviews': 'I lost all my gameplay after the update was released (',
  'label': 1},
 {'reviews': 'bad stupiud baD, O;PTION FOR CHECKPOINTS (SPOILER: THERE IS NON) NOT ADVISED FOR TWITCHY FINGERED PLAYERS OR YOU WILL DIE AND DIE AND DIE FOR 6 YEARS STRAIGHT BECAUSE ITS STUPIUD',
  'label': 0},
 {'reviews': 'i came to this game many times and now my keyboard stopped working jerking jason out...',
  'label': 1},
 {'reviews': 'shit fuck this game', 'label': 0},
 {'reviews': 'BEST GAME', 'label': 1}]

Sacafos los ejemplos, es el mismo proceso que para zero-shot:

In [236]:
# Obtenemos los resultados
resultados = list()
chunk = 15

# Pasamos las reseñas en chunks
for i in range(0, reseñas_usar.shape[0], chunk):

    # Sacamos el chunk
    reseñas_chunk = reseñas_usar.iloc[i:i+chunk]

    # Pedimos respuesta al modelo
    prediccion_completa = False
    lista_prediccion_zero_shot = obtener_predicciones_chunk_correctas(reseñas_chunk, examples_few_shot)
    print(f'Nuevas predicciones del chunk {i}: {lista_prediccion_zero_shot}')

    # Actualizamos lista
    resultados += lista_prediccion_zero_shot

# Mostramos el resultado final
print(resultados)

Nuevas predicciones del chunk 0: [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
Nuevas predicciones del chunk 15: [0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Nuevas predicciones del chunk 30: [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
Nuevas predicciones del chunk 45: [1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]


Vemos que tal le fue:

In [237]:
print(classification_report(y_true=label, y_pred=resultados))

              precision    recall  f1-score   support

           0       0.71      0.83      0.77        30
           1       0.80      0.67      0.73        30

    accuracy                           0.75        60
   macro avg       0.76      0.75      0.75        60
weighted avg       0.76      0.75      0.75        60



Y con nuestro dataset inventado:

In [238]:
# Pedimos respuesta al modelo
lista_prediccion_zero_shot = obtener_predicciones_chunk_correctas(reseñas_evaluar['reviews'])

# Mostramos el resultado final
print(lista_prediccion_zero_shot)

# Mostramos las metricas
print(classification_report(y_true=reseñas_evaluar['label'], y_pred=lista_prediccion_zero_shot))

[1, 0, 1, 1]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.67      1.00      0.80         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



Mejoró una barbaridad con respecto a zero-shot, pero aún así, Deberta y Roberta, con su fine-tunning, tienen mejores mñetricas en general (en especial, con nuestras reseñas personalizadas), pero no es descartable usar few-shot para esto. Seguramente si usamos técnicas como 'chain of thought', 'tree of thought' 'RA (reason and act)' o similares, a lo mejor obtenemos mejores resultados. Para esta práctica, zero y few shot nos basta (encima que era una extensión).

# 6. Conclusiones
Vimos que LoRA es una técnica muy útil ya que nos permite usar y personalizar (especializar) modelos transformer buenos sin necesidad de usar un hardware y recursos exagerados. Comparado con técnicas de prompt engineering, al menos, en el caso de reseñas coloquiales (muy coloquiales), resulta mucho más beneficioso realizar un entrenamiento LoRA que aplicar algunas técnicas de prompt engineering como zero o few-shot (cero o algunos ejemplos).